### process uavsar multi segment sierra line
jack tarricone
nov 16th start

In [1]:
%matplotlib inline
import isce
import os
import glob
import numpy as np
import shutil
from osgeo import gdal
import matplotlib.pyplot as plt
import rasterio as rio
from rasterio.plot import show # plotting raster data
from rasterio.plot import show_hist #histograms of raster data
from uavsar_pytools.georeference import geolocate_uavsar

2023-11-22 09:57:03,636 - rasterio.session - DEBUG - Could not import boto3, continuing with reduced functionality.


/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# get home dir
home = '/Users/jtarrico/'
os.chdir(home)
os.getcwd()

In [ ]:
# working folder name
isce_directory = 'sierra_isce_multi'
    
# path to folder
working_dir = os.path.join(home,isce_directory)

In [ ]:
# make new folder
# os.mkdir(working_dir)
print("Directory '%s' created" %isce_directory)

In [ ]:
# nativate to new folder
os.chdir(working_dir)
os.getcwd()

In [ ]:
## create subdirectories

# download
dl = 'download'
download_path = os.path.join(working_dir,dl)
# os.mkdir(download_path)

In [ ]:
# check folders were created
for file in os.listdir(working_dir):
    d = os.path.join(working_dir, file)
    if os.path.isdir(d):
        print(d)

# nativate to download folder
os.chdir(download_path)
os.getcwd()

## download

In [ ]:
# create .sh file from links

links = ['wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC.dop',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s1_1x1.llh',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s1_1x1.lkv',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s2_1x1.llh',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_01_BC_s2_1x1.lkv',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC.ann',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC_s1_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20002_001_200131_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20006_002_200212_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20009_023_200219_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20014_000_200226_L090VV_01_BC_s2_1x1.slc',
'wget https://downloaduav2.jpl.nasa.gov/Release30/sierra_17305_01/sierra_17305_20016_005_200311_L090VV_01_BC_s2_1x1.slc']

with open('downloads.sh', 'w') as f:
    for links in links:
        f.write(links)
        f.write('\n')

In [ ]:
# list files downloaded
for i in os.listdir(download_path):
    file = os.path.join(download_path, i)
    print(file)

## merge segments

add these scripts to the stripmapp folder

```pair.py``` \
```group_segments_dayhrmin_4isce.py``` \
```make_ifg_vrt_xml_isce.py``` \
```insar.py``` \
```make_slc_vrt_xml_isce.py``` \
```cat_and_interfere_py3.py``` \
```unpackFrame_UAVSAR_JPLcode.py``` 

In [ ]:
# nativate to new folder
os.chdir('./download')
os.getcwd()

In [ ]:
# read segments
! group_segments_dayhrmin_4isce.py . > images.json

In [ ]:
# generate ancillary files
! make_slc_vrt_xml_isce.py ./

In [ ]:
# create pairs .txt
! pair.py -n 1 < images.json > pairs.txt

In [ ]:
# form insar pairs at 8x2
! cat_and_interfere_py3.py -d int -r 2 -a 8 images.json pairs.txt

In [ ]:
# gen more anc files
! make_ifg_vrt_xml_isce.py -i int/

## test plot

In [ ]:
w_int = './int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.int.vrt'
coh_path = './int/20200131T1903_20200212T2211/20200131T1903_20200212T2211.cor.vrt'

# read int
ds = gdal.Open(w_int, gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

# read
coh_ds = gdal.Open(coh_path, gdal.GA_ReadOnly)
coh = coh_ds.GetRasterBand(1).ReadAsArray()
coh_ds = None

fig = plt.figure(figsize=(18, 14))

ax = fig.add_subplot(1,3,1)
ax.imshow(np.abs(igram), vmin = 0, vmax = .3, cmap = 'gray')
ax.set_title("Magnitude")
ax.set_axis_off()

ax = fig.add_subplot(1,3,2)
ax.imshow(np.angle(igram), cmap = 'jet', vmax = 3.14, vmin = -3.14)
ax.set_title("Phase")
ax.set_axis_off()

ax = fig.add_subplot(1,3,3)
ax.imshow(coh, vmin = 0 , vmax = 1, cmap = 'gray')
ax.set_title("Coherence")
ax.set_axis_off()

# Phase Unwrapping

In [ ]:
# start in downloads directory
os.chdir('/Users/jtarrico/sierra_isce_multi/')

In [ ]:
# create text file of date list
! ls --color=never int/ >> datelist.txt

In [ ]:
# print contents of that file
with open('datelist.txt') as f:
    lines = f.readlines()
    print(lines)

In [ ]:
! unwrap.py -h

In [ ]:
# run snaphu, this will take a while
! awk -F "_" '{print "unwrap.py -i int/"$1"_"$2"/"$1"_"$2".int -u int/"$1"_"$2"/"$1"_"$2".unw -c int/"$1"_"$2"/"$1"_"$2".coh -a 8 -r 2 -s download/"$1"/ -m snaphu"}' datelist.txt >> unwall.sh

In [ ]:
! sh unwall.sh

In [2]:
# start in downloads directory
os.chdir('/Users/jtarrico/sierra_isce_multi/')

In [ ]:
# list files generated in unwrapping process
!ls int_orginal/*/*

In [ ]:
# plot

# reading the multi-looked wrapped interferogram
ds = gdal.Open("int_orginal/20200131T1903_20200212T2211/20200131T1903_20200212T2211.int", gdal.GA_ReadOnly)
igram = ds.GetRasterBand(1).ReadAsArray()
ds = None

# reading the multi-looked unwrapped interferogram
ds = gdal.Open("int_orginal/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw", gdal.GA_ReadOnly)
igram_unw = ds.GetRasterBand(2).ReadAsArray()
ds = None

# reading the connected component file
ds = gdal.Open("int_orginal/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw.conncomp", gdal.GA_ReadOnly)
connected_components = ds.GetRasterBand(1).ReadAsArray()
ds = None

fig = plt.figure(figsize=(18, 16))

ax = fig.add_subplot(1,3,1)
cax=ax.imshow(np.angle(igram), cmap='jet')
ax.set_title("wrapped")
#ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[-3.14,0,3.14],orientation='horizontal')
cbar.ax.set_xticklabels(["$-\pi$",0,"$\pi$"])

ax = fig.add_subplot(1,3,2)
cax = ax.imshow(igram_unw, vmin = 2, vmax = 11, cmap = 'jet')
ax.set_title("unwrapped")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[2,0,11], orientation='horizontal')


ax = fig.add_subplot(1,3,3)
cax = ax.imshow(connected_components, cmap = 'jet')
ax.set_title("components")
ax.set_axis_off()
cbar = fig.colorbar(cax, ticks=[0,21] , orientation='horizontal')
cbar.ax.set_xticklabels([0,21])


connected_components = None

In [ ]:
! gdalinfo int_orginal/20200131T1903_20200212T2211/20200131T1903_20200212T2211.unw_snaphu.unw

## geocode

In [32]:
from pathlib import Path
from glob import glob
import os
from uavsar_pytools.convert.tiff_conversion import read_annotation, array_to_tiff
from os.path import join, basename, dirname
from rasterio.vrt import WarpedVRT

# combine llh files
def combo_llhs(data_dir: Path, ann_fp: Path):
    """
    Combines segment .llh files into a single combined .llh file and saves as a binary with .vrt.
    """
    assert data_dir.exists()

    re_llhs = {'lat':[], 'lon': [], 'height':[]}
    for llh in sorted(data_dir.glob('*.llh')):
        segment = llh.stem.split('_')[-2].replace('s','')

        data = np.fromfile(llh, np.dtype('<f'))
        lat, lon, height = data[::3], data[1::3], data[2::3]
        for key, da in zip(re_llhs.keys(), [lat, lon, height]):
            re_llhs[key].extend(da)
                               
    full = np.empty(len(re_llhs['lat'])*3, dtype='>f')
    full[0::3] = re_llhs['lat']
    full[1::3] = re_llhs['lon']
    full[2::3] = re_llhs['height']

    # read ann file
    desc = read_annotation(ann_fp)
    print('reading .ann file')

    # read in number of rows from each llh file
    nrows1 = desc[f'llh_1_2x8.set_rows']['value']
    nrows2 = desc[f'llh_2_2x8.set_rows']['value']

    # add rows for new reshaping number
    nrows_new = nrows1 + nrows2
    print('new number of rows = ',nrows_new)
    
    # read in cols, same fo rboth
    ncols = desc[f'llh_1_2x8.set_cols']['value']
    dt = np.dtype('<f')

    # create empty arrays for lat and lon
    lat_array = np.empty((nrows_new, ncols))
    lon_array = np.empty((nrows_new, ncols))
    
    # fill each layer
    lat_array = full[::3].reshape(nrows_new, ncols)
    lon_array = full[1::3].reshape(nrows_new, ncols)

    # define path to bin file
    lat_output_file = data_dir / "multi_seg.lat"
    lon_output_file = data_dir / "multi_seg.lon"

    #### Save the array to bin file
    # lat
    lat_array.tofile(lat_output_file)
    print('.lat saved')
    # lon
    lon_array.tofile(lon_output_file)
    print('.lon saved')
    

    profile = {
    'driver': 'GTiff',
    'interleave': 'band',
    'tiled': False,
    'nodata': 0,
    'width': ncols,
    'height':nrows_new,
    'count':1,
    'dtype':'float32'
    }
    
    # Save out tifs
    with rio.open(join(lat_output_file + '.tif'), 'w', **profile) as dst:
                dst.write(arr.astype(arr.dtype), 1)

    with rio.open(join(lon_output_file + '.tif'), 'w', **profile) as dst:
                dst.write(arr.astype(arr.dtype), 1)

    # Add VRT file for each tif
    tifs = glob(join(tmp_dir, '*.tif')) # list all .llh files
    for tiff in tifs: # loop to open and translate .llh to .vrt, and save .vrt using gdal
        raster_dataset = gdal.Open(tiff, gdal.GA_ReadOnly) # read in rasters
        raster = gdal.Translate(join(tmp_dir, basename(tiff).replace('.tif','.vrt')), raster_dataset, format = 'VRT', outputType = gdal.GDT_Float32)
    raster_dataset = None

    #### save vrts
    # lat
    lat_vrt = data_dir / "multi_seg.lat.vrt"
    with rio.open(lat_vrt, 'w', driver='VRT', height=nrows_new, width=ncols, count=1, dtype=dtype) as dst:
    # Add a virtual raster band to the VRT file
        dst.write(1, 
                  source_filename = lat_output_file, 
                  byte_order = "LSB", 
                  image_offset = 0, 
                  pixel_offset = 8, 
                  line_offset = ncols*8)

    # Validate the VRT file
    with rio.open(lat_vrt) as src:
        print(src.profile)

    # lon
    lon_vrt = data_dir / "multi_seg.lon.vrt"
    with rio.open(lon_vrt, 'w', driver='VRT', height=nrows_new, width=ncols, count=1, dtype=dtype) as dst:
    # Add a virtual raster band to the VRT file
        dst.write(1, 
                  source_filename = lat_output_file, 
                  byte_order = "LSB", 
                  image_offset = 0, 
                  pixel_offset = 8, 
                  line_offset = ncols*8)

    print('new .lat and .lon with .vrt have saved')


In [33]:
dir = Path('./sierra_multi_seg_llh_zach')
ann = Path('./sierra_multi_seg_llh_zach/sierra_17305_20002_001_200131_L090VV_01_BC.ann')

In [34]:
full = combo_llhs(data_dir = dir, ann_fp = ann)

reading .ann file
new number of rows =  16212
.lat saved
.lon saved
2023-11-22 10:23:09,449 - rasterio.env - DEBUG - Entering env context: <rasterio.env.Env object at 0x34f4c4d90>
2023-11-22 10:23:09,451 - rasterio.env - DEBUG - Starting outermost env
2023-11-22 10:23:09,452 - rasterio.env - DEBUG - No GDAL environment exists
2023-11-22 10:23:09,453 - rasterio.env - DEBUG - New GDAL environment <rasterio._env.GDALEnv object at 0x34f4c4550> created
2023-11-22 10:23:09,455 - rasterio._env - DEBUG - GDAL_DATA found in environment.
2023-11-22 10:23:09,456 - rasterio._env - DEBUG - PROJ_DATA found in environment.
2023-11-22 10:23:09,457 - rasterio._env - DEBUG - Started GDALEnv: self=<rasterio._env.GDALEnv object at 0x34f4c4550>.
2023-11-22 10:23:09,458 - rasterio.env - DEBUG - Entered env context: <rasterio.env.Env object at 0x34f4c4d90>
2023-11-22 10:23:09,459 - rasterio._io - DEBUG - Path: _UnparsedPath(path='sierra_multi_seg_llh_zach/multi_seg.lat.vrt'), mode: w, driver: VRT
2023-11-22 

/Users/jtarrico/miniconda/envs/isce2_env/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(


TypeError: write() got an unexpected keyword argument 'source_filename'

In [ ]:

To create a VRT file using rasterio with the specified structure, you can use the following code:

python
Copy code
import rasterio
from rasterio.vrt import VirtualRasterWarmer, WarpedVRT

# Specify the parameters
rasterXSize = 4895
rasterYSize = 8636
dataType = rasterio.float64
band = 1
sourceFilename = "lon"
byteOrder = "LSB"
imageOffset = 0
pixelOffset = 8
lineOffset = 39160

# Create a new VRT file
vrt_path = "output.vrt"
with rasterio.open(vrt_path, 'w', driver='VRT', height=rasterYSize, width=rasterXSize, count=1, dtype=dataType) as dst:
    # Add a virtual raster band to the VRT file
    dst.write_virtual_band(band, source_filename=sourceFilename, byte_order=byteOrder, image_offset=imageOffset, pixel_offset=pixelOffset, line_offset=lineOffset)

# Validate the VRT file
with rasterio.open(vrt_path) as src:
    print(src.profile)

In [ ]:
#### vrt?

In [ ]:
# Define the metadata for the virtual raster (VRT)
width, height = llh_array.shape[1], llh_array.shape[0]
count = llh_array.shape[2]  # assuming your array has multiple bands (3 in this case)
dtype = llh_array.dtype

In [ ]:
width, height, count, dtype

In [ ]:
# Define the output file path with a .vrt extension
output_file = "./sierra_multi_seg_llh_zach/multi_seg.llh.vrt"

In [ ]:
# Create a rasterio Dataset object
with rio.open(output_file, 'w', driver='VRT', height=height, width=width, count=3, dtype=dtype) as dst:
    # Write the array to the virtual raster
    dst.write(llh_array)

In [ ]:
# path to your unw.vrt
in_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/20200131T1903_20200212T2211.unw_snaphu.unw'

# path to annotation file
ann_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/sierra_17305_20002_001_200131_L090VV_01_BC.ann'

# where the .tif is being save
out_dir = '/Users/jtarrico/sierra_isce_multi/sierra_isce_multi/'

# path to lat/lon/height file
llh_fp = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/multi_seg.llh'

In [ ]:
vrt_path = '/Users/jtarrico/sierra_isce_multi/sierra_multi_seg_llh_zach/multi_seg.llh.vrt'

# Open the VRT file using rasterio
with rio.open(vrt_path) as dataset:
    # Read the raster data as a NumPy array
    vrt_array = dataset.read(1)  # Replace 1 with the band index you want to read

# Now, vrt_array contains the raster data from the VRT file
vrt_array.shape

In [ ]:
def multiseg_geolocate_uavsar(in_fp, ann_fp, out_dir, llh_fp):
    """
    Geolocates a UAVSAR SLC image with multiple segments using an array of latitudes and longitudes.
    Can be either an SLC or Look Vector. If SLC will save as a tif of real
    and a tif of complex values.
    in_fp: file path of file to geolocate
    ann_fp: file path to annotation file
    out_dir: directory to save geolocated files
    llh_fp: file path to UAVSAR lat, long, elev files for georeferencing

    returns:
    List: files that have been created
    """

    desc = read_annotation(ann_fp)
    ext = basename(in_fp).split('.')[-1]

    tmp_dir = join(out_dir, 'tmp')
    os.makedirs(tmp_dir, exist_ok=True)
        
    nrows = desc[f'llh_1_2x8.set_rows']['value']
    ncols = desc[f'llh_1_2x8.set_cols']['value']
    dt = np.dtype('<f')

    arr = np.fromfile(llh_fp, dtype = dt)
    res = {}
    res[f'llh.lat'] = arr[::3].reshape(nrows, ncols)
    res[f'llh.long'] = arr[1::3].reshape(nrows, ncols)
    res[f'llh.dem'] = arr[2::3].reshape(nrows, ncols)

    profile = {
    'driver': 'GTiff',
    'interleave': 'band',
    'tiled': False,
    'nodata': 0,
    'width': ncols,
    'height':nrows,
    'count':1,
    'dtype':'float32'
    }
    
    # Save out tifs
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.")
        for name, arr in res.items():
            with rio.open(join(tmp_dir, name + '.tif'), 'w', **profile) as dst:
                dst.write(arr.astype(arr.dtype), 1)

    # Add VRT file for each tif
    tifs = glob(join(tmp_dir, '*.tif')) # list all .llh files
    for tiff in tifs: # loop to open and translate .llh to .vrt, and save .vrt using gdal
        raster_dataset = gdal.Open(tiff, gdal.GA_ReadOnly) # read in rasters
        raster = gdal.Translate(join(tmp_dir, basename(tiff).replace('.tif','.vrt')), raster_dataset, format = 'VRT', outputType = gdal.GDT_Float32)
    raster_dataset = None

    vrts = glob(join(tmp_dir, '*.vrt'))
    latf = [f for f in vrts if basename(f) == 'llh.lat.vrt'][0]
    longf = [f for f in vrts if basename(f) == 'llh.long.vrt'][0]

    profile = {
        'driver': 'GTiff',
        'interleave': 'band',
        'tiled': False,
        'nodata': 0,
        'width': ncols,
        'height':nrows,
        'count':1,
        'dtype':'float32'
        }

    if ext == 'slc':
        spacing = in_fp.replace(f'.{ext}','')[-3:]
        nrows = desc[f'{ext}_1_{spacing} rows']['value']
        ncols = desc[f'{ext}_1_{spacing} columns']['value']
        dtype = np.complex64
        arr = np.fromfile(in_fp, dtype = dtype).reshape(nrows, ncols)
        d_arrs = {}
        d_arrs['real'] = arr.real
        d_arrs['imag'] = arr.imag

    elif ext == 'lkv':
        spacing = in_fp.replace(f'.{ext}','')[-3:]
        nrows = desc[f'{ext}_1_{spacing} rows']['value']
        ncols = desc[f'{ext}_1_{spacing} columns']['value']
        dtype = np.dtype('<f')
        arr = np.fromfile(in_fp, dtype = dtype)
        d_arrs = {}
        d_arrs[f'y'] = arr[::3].reshape(nrows, ncols)
        d_arrs[f'x'] = arr[1::3].reshape(nrows, ncols)
        d_arrs[f'z'] = arr[2::3].reshape(nrows, ncols)

    elif ext == 'vrt':
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.")
            second_ext = basename(in_fp).split('.')[-2]
            if second_ext == 'unw':
                with rio.open(in_fp) as src:
                    # 1st band is amplitude, 2nd band is unwrapped phase
                    arr = src.read(2)
            else:
                with rio.open(in_fp) as src:
                    arr = src.read(1)
            d_arrs = {}
            ext = second_ext
            d_arrs[second_ext] = arr

    # Save out tifs
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.")
        for name, arr in d_arrs.items():
            with rio.open(join(tmp_dir, basename(in_fp) + f'.{name}.tif'), 'w', **profile) as dst:
                dst.write(arr.astype(arr.dtype), 1)

        tifs = glob(join(tmp_dir, f'*{ext}*.tif')) # list all .ext files
        for tiff in tifs: # loop to open and translate .ext to .vrt, and save .vrt using gdal
            raster_dataset = gdal.Open(tiff, gdal.GA_ReadOnly) # read in rasters
            raster = gdal.Translate(join(tmp_dir, basename(tiff).replace('.tif','.vrt')), raster_dataset, format = 'VRT', outputType = gdal.GDT_Float64)
        raster_dataset = None

        vrts = glob(join(tmp_dir, f'*{ext}*.vrt'))
        res_f = []
        for f in vrts:
            out_f = join(out_dir, basename(f).replace('vrt','tif'))
            geocodeUsingGdalWarp(infile = f,
                                latfile = latf,
                                lonfile = longf,
                                outfile = out_f,
                                spacing=[.00005556,.00005556])

            res_f.append(out_f)

        if ext == 'unw':
            print('Ignore the error message: Unable to compute bounds. It is related\n\
                to the pixels created by the conversion along the edge of topography.\n\
                Error message is known and should not be an issue.')
        
    shutil.rmtree(tmp_dir)

    return res_f

In [ ]:
# run geolocate function
geolocated_data = geolocate_uavsar(in_fp = in_fp, ann_fp = ann_fp, out_dir = out_dir, llh_fp = llh_fp)
print(geolocated_data[0])

In [ ]:
# path to your unw.vrt
in_fp = 'igrams/20200226T2253_20200311T1852/20200226T2253_20200311T1852.coh.vrt'

# path to annotation file
ann_fp = '/Users/jtarrico/sierra_isce/SLC/20200226T2253/sierra_17305_20014_000_200226_L090VV_01_BC.ann'

# where the .tif is being save
out_dir = '/Users/jtarrico/sierra_isce/geocode_test'

# path to lat/lon/height file
llh_fp = '/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC_s1_2x8.llh'

# coh_geo = geolocate_uavsar(in_fp = in_fp, ann_fp = ann_fp, out_dir = out_dir, llh_fp = llh_fp)
# print(coh_geo[0])

In [ ]:
# path to your unw.vrt
in_fp = 'igrams/20200226T2253_20200311T1852/20200226T2253_20200311T1852.unw_snaphu.unw.conncomp.vrt'

# path to annotation file
ann_fp = '/Users/jtarrico/sierra_isce/SLC/20200226T2253/sierra_17305_20014_000_200226_L090VV_01_BC.ann'

# where the .tif is being save
out_dir = '/Users/jtarrico/sierra_isce/geocode_test'

# path to lat/lon/height file
llh_fp = '/Users/jtarrico/sierra_isce/download/sierra_17305_01_BC_s1_2x8.llh'

concomp_geo = geolocate_uavsar(in_fp = in_fp, ann_fp = ann_fp, out_dir = out_dir, llh_fp = llh_fp)
print(concomp_geo[0])

In [ ]:
# Plot geolocated unwrapped phase data!
with rio.open(geolocated_data[0]) as src:
    plt.subplots(1, figsize=(15, 15))
    show(src, vmin = -3, vmax = 3) 